In [0]:

-- Graph driver node's CPU, RAM, Network, Disk
WITH node AS (
    SELECT *
    FROM system.compute.node_types
),
clusters AS (
    SELECT DISTINCT
        c.cluster_id,
        c.cluster_name
    FROM system.compute.clusters AS c
    WHERE  COALESCE(CAST(:cluster_name AS STRING), '') = ''
        OR INSTR(CAST(:cluster_name AS STRING), c.cluster_name) > 0
        OR INSTR(CAST(:cluster_name AS STRING), '[ANY]') > 0
),
timeline AS (
    SELECT
        n.account_id,
        n.workspace_id,
        n.cluster_id,
        n.instance_id,
        n.start_time,
        n.end_time,
        n.driver,
        n.cpu_user_percent,
        n.cpu_system_percent,
        n.cpu_wait_percent,
        n.mem_used_percent,
        n.mem_swap_percent,
        n.node_type,      
        n.network_sent_bytes,
        n.network_received_bytes,
        p.*
    FROM system.compute.node_timeline AS n,    
    LATERAL POSEXPLODE(n.disk_free_bytes_per_mount_point) AS p
    WHERE driver = TRUE               
        AND (n.start_time > :from)
        AND (n.start_time < :to)
        AND (n.cluster_id IN (SELECT c.cluster_id FROM clusters as c))
    ORDER BY n.instance_id, n.start_time desc
),
timeline_node AS (
    SELECT
        timeline.*,
        node.node_type,
        node.core_count,
        node.memory_mb,
        node.gpu_count,
        timeline.cpu_system_percent + timeline.cpu_user_percent AS cpu_percent,
        (timeline.cpu_system_percent + timeline.cpu_user_percent) * core_count / 100 AS cpu_cores_used,
        mem_used_percent * memory_mb / 100 AS mem_mb_used
    FROM timeline
    LEFT JOIN node
        ON node.account_id = timeline.account_id
        AND node.node_type = timeline.node_type
)
SELECT
    timeline_node.account_id,
    timeline_node.cluster_id,
    timeline_node.instance_id,
    timeline_node.start_time,
    timeline_node.end_time,
    timeline_node.driver,
    timeline_node.cpu_user_percent,
    timeline_node.cpu_system_percent,
    timeline_node.cpu_wait_percent,
    timeline_node.cpu_percent,
    timeline_node.cpu_cores_used,
    timeline_node.mem_used_percent,
    timeline_node.mem_mb_used,
    timeline_node.mem_swap_percent,
    timeline_node.gpu_count,
    timeline_node.core_count,
    timeline_node.memory_mb,
    timeline_node.network_sent_bytes,
    timeline_node.network_received_bytes,
    timeline_node.key,
    timeline_node.value
FROM timeline_node;



Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.